# Leaflet cluster map of talk locations

(c) 2016-2017 R. Stuart Geiger, released under the MIT license

Run this from the _talks/ directory, which contains .md files of all your talks. This scrapes the location YAML field from each .md file, geolocates it with geopy/Nominatim, and uses the getorg library to output data, HTML, and Javascript for a standalone cluster map.

Requires: glob, getorg, geopy

In [1]:
#!pip install getorg python-frontmatter

In [2]:
import glob
import getorg
from geopy import Nominatim

Iywidgets and ipyleaflet support disabled. You must be in a Jupyter notebook to use this feature.
Error raised:
No module named 'ipyleaflet'
Check that you have enabled ipyleaflet in Jupyter with:
    jupyter nbextension enable --py ipyleaflet


In [3]:
g = glob.glob("_talks/*.md")

In [4]:
geocoder = Nominatim(user_agent="my-application1") # small hack, as `user_agent="my-application"` (default) returns an error
location_dict = {}
location = ""
permalink = ""
title = ""
venue = ""

In [5]:
count=0
for file in g:
    with open(file, 'r') as f:
        lines = f.read()
        if lines.find('location: "') > 1:
            loc_start = lines.find('location: "') + 11
            lines_trim = lines[loc_start:]
            loc_end = lines_trim.find('"')
            location = lines_trim[:loc_end]
        if lines.find('title: "') > 1:
            loc_start = lines.find('title: "') + 8
            lines_trim = lines[loc_start:]
            loc_end = lines_trim.find('"')
            title = lines_trim[:loc_end]
        if lines.find('venue: "') > 1:
            loc_start = lines.find('venue: "') + 8
            lines_trim = lines[loc_start:]
            loc_end = lines_trim.find('"')
            venue = lines_trim[:loc_end]

        key = str(title + " | " + venue + ", " + location)

        if location.endswith("(online)"): # special case for online talks
            location = location.rstrip(" (online)")
        
        location_dict[key] = geocoder.geocode(location, timeout=60)
        
        print(key, "\n", location_dict[key], "\n")
    
    count += 1
print(count, "talks")

Hysteresis of the Antarctic Ice Sheet | EGU General Assembly 2019, Vienna, Austria 
 Wien, 1010, Österreich 

The Hysteresis of the Antarctic Ice Sheet | InTENSE Workshop, Bad Belzig, Germany 
 Bad Belzig, Potsdam-Mittelmark, Brandenburg, 14806, Deutschland 

Hysteresis behaviour of the Antarctic Ice Sheet with the Parallel Ice Sheet Model | EGU General Assembly 2018, Vienna, Austria 
 Wien, 1010, Österreich 

Tipping of the Antarctic Ice Sheet influenced by Greenland ice loss | Potsdam Institute for Climate Impact Research, Potsdam, Germany 
 Potsdam, Brandenburg, Deutschland 

The Hysteresis of the Antarctic Ice Sheet | Potsdam Institute for Climate Impact Research, Potsdam, Germany (online) 
 Potsdam, Brandenburg, Deutschland 

Antarctic ice dynamics - from deep past to deep future | EGU General Assembly 2020, Vienna, Austria (online) 
 Wien, 1010, Österreich 

Hysteresis Behavior of the Antarctic Ice Sheet | Potsdam Institute for Climate Impact Research, Potsdam, Germany 
 Potsdam,

In [6]:
# remove old org-locations.js first, otherwise changed entries just get appended
import os
os.remove("talkmap/org-locations.js")

In [7]:
m = getorg.orgmap.create_map_obj()
getorg.orgmap.output_html_cluster_map(location_dict, folder_name="talkmap/", hashed_usernames=False)

'Written map to talkmap/'

In [8]:
# some files include manual changes but are overwitten by getorg, like map.html and leaflet_dist/screen.css
# thus a backup version is kept in restore/ and copied to replace the newly created ones
import shutil
print('copying files from restore/ directory')
shutil.copytree("./talkmap/restore/","./talkmap/", dirs_exist_ok=True)

copying files from restore/ directory


'./talkmap/'